In [13]:
import importlib
import sentencepiece
import torch
import random
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import json
import Automation.automation as automation
from Automation.dag_auto import fill_DAG
import pandas as pd

# Library
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from cycler import cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sklearn.ensemble import RandomForestRegressor
from transformers import AutoTokenizer, AutoModelForCausalLM
import Automation.automation as automation

from Automation.sdscm import plot_dag_from_sample_space, sample_sequences, format_sequences_as_dataframe, sample_counterfactual_sequences

def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue 
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                raise ValueError(f"Invalid JSON in line: {line}\nError: {e}")
    return data



def load_candidates(path="candidates_set.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data
candidates_set =load_candidates('ihdp_candidates_set.json')

import json

def save_candidates(candidates_set, path="ihdp_candidates_set.json"):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(candidates_set, f, ensure_ascii=False, indent=4)
    print(f"Saved to {path}")


In [14]:
dag_json = load_jsonl("/home/zeyuan/Capstone/data/dags_runs (1).jsonl")
new_df = pd.read_csv("./data/ihdp_full_cleaned.csv")
new_df =new_df.drop("Unnamed: 0", axis = 1)
new_df =new_df.drop("realization", axis = 1)
new_df =new_df.drop("ycf", axis = 1)
new_df["yf"] = pd.to_numeric(new_df["yf"], errors="coerce").fillna(0).astype(int)
new_df["yf"] = new_df["yf"].clip(lower=0, upper=10)
new_df["yf"].unique()

array([ 1,  3,  2,  0,  6,  5,  4, 10,  7,  9,  8])

In [15]:
# Original llm
model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",  
    device_map="auto"          
    
)
model.eval()

def call_Qwen(cur_prompt, stop=None):
    messages = [
        {"role": "user", "content": cur_prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)

    inputs = tokenizer(text, return_tensors="pt").to(model.device)



    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        temperature=0.4,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )


    response = tokenizer.decode(outputs, skip_special_tokens=True)

    return response.strip()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
#New dataset with discrete candidatews
description = """
The Infant Health and Development Program (IHDP) dataset contains detailed information about infants and their families who participated in a randomized intervention study. 
The dataset includes both demographic and socio-economic characteristics of the mothers (such as age, race, marital status, and education) as well as characteristics of the children (such as birth weight, gestational age, and health risk factors). 
In the study, some infants were randomly assigned to receive an early childhood educational intervention, while others served as a control group. 
The dataset also contains outcome variables measuring cognitive test scores at later ages, allowing researchers to study treatment effects under realistic confounding conditions. 
Several continuous variables have been discretized into categorical levels (e.g., low, mid, high) to represent grouped ranges of demographic or health-related measurements for modeling purposes.
"""
few_examples = [
    "Sentence: The mother is a young adult and is recorded as having completed a relatively low level of formal education.",
    
    "Sentence: The child received the intervention treatment and participated in the early childhood development program provided by the study.",
    
    "Sentence: The infant was born with a low birth weight, indicating an elevated level of perinatal health risk at the time of delivery."
]

candidates_set = automation.build_long_candidates_set(tokenizer,model,new_df,description=description,few_examples=few_examples)
for i in candidates_set.keys():
    print(f"{i}: {candidates_set[i]}\n")
save_candidates(candidates_set)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Raw output] The infant was born with a normal birth weight, placing them within the typical range for healthy newborns. 

Okay, let's tackle this query step by step. The user wants a descriptive sentence about
[True output]The infant was born with a normal birth weight, placing them within the typical range for healthy newborns.
[Raw output] The infant was classified as having a low birth weight, which may indicate potential health risks during pregnancy or at birth. 

Okay, let's tackle this query step by step. The user wants a descriptive
[True output]The infant was classified as having a low birth weight, which may indicate potential health risks during pregnancy or at birth.
[Raw output] The infant was born with a high birth weight, which may indicate potential health advantages or complications depending on other factors. 

Okay, let's tackle this query step by step. First, I need to
[True output]The infant was born with a high birth weight, which may indicate potential health ad

KeyboardInterrupt: 

In [ ]:
import importlib
import sentencepiece
import torch
import random
from transformers import PegasusForConditionalGeneration, PegasusTokenizer



# ----------------------
# Load Pegasus model
# ----------------------
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


# ----------------------
# Pegasus paraphrase function
# ----------------------
def paraphrase_sentence(text, num_return_sequences=5, num_beams=5, max_length=60, padding= "longest"):

    batch = tokenizer(
        [text],
        truncation=True,
        padding= padding,
        max_length=60,
        return_tensors="pt"
    ).to(torch_device)

    outputs = model.generate(
        **batch,
        max_length=max_length,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        temperature=1.5
    )

    paraphrased = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [p.strip() for p in paraphrased]


# ----------------------
# Generate N new candidate sets
# ----------------------
def generate_n_candidate_sets(candidates_set, n=5,num_beams=5, padding= "longest", max_length=60 ):
    """
    candidates_set: dict[str, list[str]]
        e.g. {"Survived": ["The passenger died.", "The passenger survived."], ...}
    n: how many candidate_sets you want (also = how many paraphrases per sentence)
    """

    paraphrased = {}

    for var, sentences in candidates_set.items():
        var_paras = []
        for sent in sentences:
            paras = paraphrase_sentence(sent, num_return_sequences=n,padding =padding,num_beams =num_beams, max_length =max_length)

            if len(paras) < n:
                while len(paras) < n:
                    paras.append(random.choice(paras))
            var_paras.append(paras)
        paraphrased[var] = var_paras

    all_sets = []

    for k in range(n):  
        new_set = {}
        for var, var_paras in paraphrased.items():
            new_sents = [paras[k] for paras in var_paras]
            new_set[var] = new_sents
        all_sets.append(new_set)

    return all_sets

In [ ]:
import json

def save_candidates(candidates_set, path="ihdp_candidates_set.json"):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(candidates_set, f, ensure_ascii=False, indent=4)
    print(f"Saved to {path}")

In [ ]:
import json
def load_candidates(path="candidates_set.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data
candidates_set =load_candidates('ihdp_candidates_set.json')



In [ ]:
generated_sets_long = generate_n_candidate_sets(candidates_set, n= 10,num_beams =10)
generated_sets_Med = generate_n_candidate_sets(candidates_set, n= 10,num_beams =10, padding = "max_length",max_length =15)
generated_sets_short = generate_n_candidate_sets(candidates_set, n= 10, num_beams= 10, padding = "max_length",max_length =8)

for i, s in enumerate(generated_sets_long):
    print(f"\n=== Candidate Set {i+1} ===")
    print(s)

for i, s in enumerate(generated_sets_Med):
    print(f"\n=== Candidate Set {i+1} ===")
    print(s)

for i, s in enumerate(generated_sets_short):
    print(f"\n=== Candidate Set {i+1} ===")
    print(s)

In [ ]:
def parse_dag(dag_json):
  
    parent_indices = dag_json["parent_indices"]
    var_names = dag_json["var_names"]

    # ① index → parent indices
    parents_idx_by_child = {
        i: list(parent_indices[i] or [])
        for i in range(len(var_names))
    }

    # name → parent indices
    parents_idx_by_name = {
        var_names[i]: parents_idx_by_child[i]
        for i in range(len(var_names))
    }

    exogenous_by_name = {
        name: (len(parents_idx_by_name[name]) == 0)
        for name in var_names
    }

    return parents_idx_by_child, exogenous_by_name


In [ ]:
all_sets = [
    ("long", generated_sets_long),
    ("med", generated_sets_Med),
    ("short", generated_sets_short)
]

for tag, sets in all_sets:
    for i, s in enumerate(sets):
        save_candidates(s, path=f"ihdp_ready_{tag}_{i}.json")


In [ ]:
init_framework_from_df = automation.init_framework_from_df(new_df,save_path= "ihdp_framework.json")
for x in range(0,20, 2):
    dag = dag_json[x]
    parents_idx_by_name = dag["parents_idx_by_name"]
    exogenous_by_name = dag["exogenous_by_name"]
    types = ["long", "med", "short"]
    for t in types:
        dag_idx = x//2
        input_candidate_file = f"ihdp_ready_{t}_{dag_idx}.json"
        input_candidate = load_candidates(input_candidate_file)

        with_candidates_file = f"ihdp_with_candidates_{dag_idx}_{t}.json"
        automation.fill_candidate_sets(
            "ihdp_framework.json",
            input_candidate,
            with_candidates_file,
        )
        
        print(with_candidates_file)
        output_seq_file = f"ihdp_complete_{dag_idx}_{t}.json"
        print(exogenous_by_name)
        print(len(exogenous_by_name))
        fill_DAG(
            with_candidates_file,
            parents_idx_by_name,
            exogenous_by_name,
            output_seq_file,
        )

    print("finish")
